# **400m Career Data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
%matplotlib inline
from datetime import datetime
import re
from nssstats.plots import std_plot
from nssstats.plots import iqr_plot
from nssstats.plots import quadrant_plot, half_plot
from ipywidgets import interact, FloatSlider
from nssstats.cm import cm_analysis
from sklearn.model_selection import train_test_split
from scipy.stats import probplot
from scipy.stats import t, sem
from scipy.stats import chi2
from statsmodels.stats.proportion import proportion_confint
import statsmodels.formula.api as sm

In [ ]:
sprinters = pd.read_csv("Worlds_Fastest_Sprinters_Stats.csv")

# **Data** **Basics**

In [ ]:
sprinters.head()

In [ ]:
sprinters.info

In [ ]:
sprinters.shape

In [ ]:
print(sprinters.dtypes)


In [ ]:
sprinters.describe()


In [ ]:
sprinters.corr


In [ ]:
sprinters.isnull().sum()


# **General** **EDA**

Let's make a column for the total PR time

In [ ]:
sprinters['Total_Time_PRs'] = round(sprinters['100_PR'] + sprinters['200_PR'] + sprinters['400_PR'],2)
sprinters.head()

Let's make a column for the total career average time

In [ ]:
sprinters['Total_Time_SB_Avg'] = round(sprinters['Avg_Season_Best_100m'] + sprinters['Avg_Season_Best_200m'] + sprinters['Avg_Season_Best_400m'],2)
sprinters.head()

Let's make a column for the actual age of the athletes

In [ ]:
# Ensure the DOB column is in datetime format
sprinters['DOB'] = pd.to_datetime(sprinters['DOB'], errors='coerce')  # Coerce will handle invalid dates as NaT

# Get the current year
current_year = datetime.now().year

# Calculate the age by subtracting the birth year from the current year
sprinters['Age'] = current_year - sprinters['DOB'].dt.year

# Display the updated DataFrame with the new 'Age' column
print(sprinters[['DOB', 'Age']].head())


Let's add the sprinter's photo to the database by merging it with the photo csv

In [ ]:
sprinter_photo = pd.read_csv("Sprinter_Photo.csv")

In [ ]:
sprinter_photo.head(3)

In [ ]:
sprinters = pd.merge(sprinters, sprinter_photo, on=['Athlete'],how='left')
sprinters.head(3)

Let's add second database to join number of season to each events dataframe (For Years Competed in each event).

In [ ]:
sprinters_df2 = pd.read_csv("Worlds_Fastest_Sprinters_Master_List_Yearly_Progression.csv")
sprinters_df2.head(3)

In [ ]:
yrs_competed_400m = sprinters_df2[sprinters_df2['Event'] == '400m'].groupby('Athlete').size().reset_index(name='Years')

In [ ]:
yrs_competed_400m = yrs_competed_400m.sort_values(by='Years', ascending=False)
yrs_competed_400m.head()

Let's add a third database which incorportae's every race in each athletes's career.

In [ ]:
sprinters_df3 = pd.read_csv("Sprinter_Career.csv")
sprinters_df3.head(3)

In [ ]:
All_400m_Races = sprinters_df3[sprinters_df3['Event'] == '400m']
All_400m_Races.head(3)

Let's drop all the races that were DNS, DNF, or DQ

In [ ]:
All_400m_Races = All_400m_Races[~All_400m_Races['Time'].isin(['DNS', 'DNF', 'DQ'])]
All_400m_Races.head(3)

Let's make sure that the time column is now a numeric datatype

In [ ]:
All_400m_Races['Time'] = pd.to_numeric(All_400m_Races['Time'], errors='coerce')

Let's drop all the Indoor marks

In [ ]:
All_400m_Races = All_400m_Races[All_400m_Races['Meet_Type'] != 'Indoor']
All_400m_Races.head(3)

Let's drop times that aren't legal from the dataframe

In [ ]:
All_400m_Races = All_400m_Races[All_400m_Races['Legal'] != 'NO']
All_400m_Races.head(3)

Let's look at a couple of visualizations of this dataframe

In [ ]:
All_400m_Races['Time'].hist();

In [ ]:
probplot(All_400m_Races['Time'], plot=plt);

In [ ]:
plt.figure(figsize = (10,6))

std_plot(All_100m_Races['Time'], edgecolor = 'black', linewidth = 2)

In [ ]:
plt.figure(figsize = (10,6))

iqr_plot(All_400m_Races['Time'], bins = 25, edgecolor = 'black', linewidth = 2)

In [ ]:
plt.figure(figsize = (10,6))
sns.boxplot(x = All_400m_Races['Time']);

In [ ]:
sns.boxplot(data = All_400m_Races, y = 'Athlete', x = 'Time')
plt.xticks([0,1], ['', ''])
plt.xlabel('400m Time')
plt.title('400m Times Over Entire Career');

Let's get the Caeer Average for Each Athlete

In [ ]:
Career_average_400m = All_400m_Races.groupby('Athlete')['Time'].mean().reset_index(name='Career_Avg_400m')

In [ ]:
#All_400m_Races = pd.merge(All_400m_Races, Career_average_400m, on=['Athlete'],how='left')
#All_400m_Races.head(3)

Total Races for Each Athlete

In [ ]:
athlete_race_count_400m = All_400m_Races.groupby('Athlete').size().reset_index(name='total_races_400m')
athlete_race_count_400m.head(3)

Number of races for each athlete by year

In [ ]:
#athlete_race_count_per_year_400m = hundred_meter_races.groupby(['Athlete', 'Year']).size().reset_index(name='races_per_year_100m')
#athlete_race_count_per_year_400m.head(3)

Meerge Data Season and total races data next

In [ ]:
seasons_and_races_400m = pd.merge(athlete_race_count_400m, yrs_competed_400m, on=['Athlete'],how='left')
seasons_and_races_400m.head(3)

Let's add the career average for each athlete

In [ ]:
seasons_and_races_400m = pd.merge(seasons_and_races_400m, Career_average_400m, on=['Athlete'],how='left')
seasons_and_races_400m.head(3)

In [ ]:
seasons_and_races_400m['Avg_Races_Year_400m'] = round(seasons_and_races_400m['total_races_400m'] / seasons_and_races_400m['Years'],2)
sprinters.head()

In [ ]:
df_400m= sprinters[['Athlete', 'Country','Continent','Status', 'DOB','Year Born','Month Born','Decade Born','Avg_Season_Best_400m','400_PR','T25_400_All_Time_Rank','T25_400_AT_RK_NUM']]
df_400m

In [ ]:
df_400m['SB_Avg_400m_PR_Diff'] = round(df_400m['Avg_Season_Best_400m'] - df_400m['400_PR'],2)
df_400m.head(3)

In [ ]:
df_400m = pd.merge(df_400m, seasons_and_races_400m, on=['Athlete'],how='left')
df_400m.head(3)

In [ ]:
df_400m = df_400m.rename(columns={'Years': 'Years_Competed_400m'})
df_400m.head(3)

In [ ]:
df_400m['Career_Avg_400m_PR_Diff'] = round(df_400m['Career_Avg_400m'] - df_400m['400_PR'],2)
df_400m.head(3)

In [ ]:
#df_400m = df_400m.sort_values(by='Avg Season Best 400m', ascending=False)
#df_400m

In [ ]:
#df_400m = df_400m.sort_values(by='Avg_Season_Best_400m')
#df_400m

In [ ]:
df_400m = df_400m.sort_values(by='Career_Avg_400m')
df_400m

**Top 5 400m Times (PRs)**



1.   Wayne Van Niekerk: 43.03 (11 Seasons)
2.   Michael Johnson: 43.18 (14 Seasons)
3.   Butch Reynolds: 43.29 ( Seasons)
4.   Quincy Hall : 43.40 ( Seasons)
5.   Matthew Hudson-Smith: 43.44 ( Seasons)








**Top 5 Career Season Best Averages in the 400m (Rounded to the nearest hundreth)**



1.   Larry James: 43.97 (1 Season)
2.   Michael Johnson: 44.22 (14 Seasons)
3.   Steven Gardiner: 44.39 (10 Seasons)
4.   Lee Evans : 44.41 (2 Seasons)
5.   Wayne Van Niekerk: 44.48 (11 Seasons)








**Top 5 Career Averages in the 400m (Rounded to the nearest hundreth)**



1. Michael Johnson: 44.50 (14 Seasons)
2. LaShawn Merritt: 44.89 (17 Seasons)  
3. Lee Evans: 45.04 (2 Seasons)
4. Steve Lewis: 45.04 (13 Seasons)
5. Kirani James: 45.10 (17 Seasons)








# ***EDA Visualizations***

In [ ]:
# @title Year Born 400m


df_400m['Year Born'].plot(kind='hist', bins=20, title='Year Born')
plt.gca().spines[['top', 'right',]].set_visible(False)

#plt.savefig('year_born_400.png', format='png', dpi=300)
#plt.savefig('year_born_400.jpg', format='jpg', dpi=300)

In [ ]:
plt.figure(figsize = (10,6))

df_400m.groupby('Country')['Athlete'].count().plot(kind = 'bar')
plt.title('400m Sprinters By 100m BY Country')
plt.ylabel('count')
plt.xticks(rotation = 0);

In [ ]:
plt.figure(figsize = (10,6))

df_400m.groupby('Continent')['Athlete'].count().plot(kind = 'bar')
plt.title('400m Sprinters By 100m BY Continent')
plt.ylabel('count')
plt.xticks(rotation = 0);

In [ ]:
# @title Year Born vs Avg Season Best 400m

df_400m.plot(kind='scatter', x='Year Born', y='Avg_Season_Best_400m', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

#plt.savefig('year_born_SB_avg_400.png', format='png', dpi=300)
#plt.savefig('year_born_SB_avg_400.jpg', format='jpg', dpi=300)

In [ ]:
# @title Year Born vs Career Avg  400m

df_400m.plot(kind='scatter', x='Year Born', y='Career_Avg_400m', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

#plt.savefig('year_born_Career_avg_400.png', format='png', dpi=300)
#plt.savefig('year_born_Career_avg_400.jpg', format='jpg', dpi=300)

In [ ]:
# @title Avg Season Best 400m vs 400_PR

df_400m.plot(kind='scatter', x='Avg_Season_Best_400m', y='400_PR', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

#plt.savefig('SB_avg_400_pr.png', format='png', dpi=300)
#plt.savefig('SB_avg_400_pr.jpg', format='jpg', dpi=300)

Version 2

In [ ]:
df_400m = px.data.iris() # iris is a pandas DataFrame
fig = px.scatter(df_400m, x="Avg_Season_Best_400m", y="400_PR")
fig.show()

#fig.write_html("SB_400m_avg_vs_pr_scatter.html")
#fig.write_image("SB_400m_avg_vs_pr_scatter.svg")

Version 3

In [ ]:
df_400m = px.data.iris()
fig = px.scatter(df_400m, x="Avg_Season_Best_400m", y="400_PR", color="Country",
                 size='Avg_Season_Best_400m', hover_data=['400_PR'])
fig.show()

#fig.write_html("SB_400m_avg_vs_pr_scatter_2.html")
#fig.write_image("SB_400m_avg_vs_pr_scatter_2.svg")

Version 4 (With Error Bars)

In [ ]:
df_400m = px.data.iris()
df_400m["e"] = df_400m["Avg_Season_Best_400m"]/100
fig = px.scatter(df_400m, x="Avg_Season_Best_400m", y="400_PR", color="Country",
                 error_x="e", error_y="e")
fig.show()

#fig.write_html("SB_400m_avg_vs_pr_scatter_3.html")
#fig.write_image("SB_400m_avg_vs_pr_scatter_3.svg")

Version 5 (Using Dash)

In [ ]:


app = Dash(__name__)


app.layout = html.Div([
    html.H4('Career Average 400m vs 400m PR'),
    dcc.Graph(id="scatter-plot"),
    html.P("Filter by Career Season Best Average 400m:"),
    dcc.RangeSlider(
        id='range-slider',
        min=0, max=2.5, step=0.1,
        marks={0: '0', 2.5: '2.5'},
        value=[0.5, 2]
    ),
])


@app.callback(
    Output("scatter-plot", "figure"),
    Input("range-slider", "value"))
def update_bar_chart(slider_range):
    df_400m = px.data.iris() # replace with your own data source
    low, high = slider_range
    mask = (df_400m['Avg_Season_Best_400m'] > low) & (df_400m['Avg_Season_Best_400m'] < high)
    fig = px.scatter(
        df[mask], x="Avg_Season_Best_400m", y="400m_PR",
        color="Country", size='Avg Season Best 400m',
        hover_data=['400m_PR'])
    return fig


app.run_server(debug=True)

#fig.write_html("SB_400m_avg_vs_pr_scatter_dash.html")
#fig.write_image("SB_400m_avg_vs_pr_scatter_dash.svg")

In [ ]:
# @title Career Avg 400m vs 400_PR

df_400m.plot(kind='scatter', x='Career_Avg_400m', y='400_PR', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

#plt.savefig('career_avg_400__vs_pr.png', format='png', dpi=300)
#plt.savefig('career_avg_400_vs_pr.jpg', format='jpg', dpi=300)

Let's look at this more in depth

In [ ]:
plt.figure(figsize = (12,8))

sns.scatterplot(data = df_400m, x = 'Career_Avg_400m', y = '100_PR',
                hue = 'cylinders', palette = 'Blues', edgecolor = 'black'
               )
plt.title('Career Average 400m vs. 400m PR');

In [ ]:
quadrant_plot(df_400m['Career_Avg_400m'], df_400m['400_PR'], labels = ['Career Average 400m', '400m PR'], figsize = (12,8))

In [ ]:
quadrant_plot(df_400m['Career_Avg_400m'],
              df_400m['400_PR'],
              labels = ['Career Average 400m', '400m PR'],
              quadrant = 4,
              figsize = (12,8))

In [ ]:
quadrant_plot(df_400m['Career_Avg_400m'],
              df_400m['400_PR'],
              labels = ['Career Average 400m', '400m PR'],
              quadrant = 2,
              figsize = (12,8))

In [ ]:
quadrant_plot(df_400m['Career_Avg_400m'],
              df_400m['400_PR'],
              labels = ['Career Average 400m', '400m PR'],
              quadrant = 3,
              figsize = (12,8))

In [ ]:
quadrant_plot(df_400m['Career_Avg_400m'],
              df_400m['400_PR'],
              labels = ['Career Average 400m', '400m PR'],
              quadrant = 1,
              figsize = (12,8))

In [ ]:
half_plot(df_400m['Career_Avg_400m'], df_400m['400_PR'], labels = ['Career_Avg_400m', '400_PR'],
          figsize = (12,8), half = 'left')

In [ ]:
half_plot(df_400m['Career_Avg_400m'], df_400m['400_PR'], labels = ['Career_Avg_400m', '400_PR'],
          figsize = (12,8), half = 'right')

In [ ]:
fig, ax = plt.subplots(figsize = (12,8))
df_400m.plot(kind = 'scatter', x = 'Career_Avg_400m', y = '400_PR', ax = ax)

x = np.linspace(df_400m['Career_Avg_400m'].min(), df_400m['400_PR'].max(), 100)
z = np.polyfit(df_400m['Career_Avg_400m'], df_400m['400_PR'], 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

plt.title('Career Average 400m vs. 400m PR');

Heatmaps

In [ ]:
# Calculate the average 400m PR per country
avg_400m_pr_per_country = df_400m.groupby('Country')['400_PR'].mean().reset_index()

fig = px.choropleth(avg_400m_pr_per_country,
                    locations="Country",
                    locationmode='country names',
                    color="400_PR",
                    hover_name="Country",
                    color_continuous_scale="Viridis",
                    title="Average 400m Personal Records by Country (Faster = Darker)")

fig.show()
#fig.write_html("avg_400m_PR_country_map.html")
#fig.write_image("avg_400m_PR_country_map.svg")

In [ ]:
# Extract birth months from DOB
sprinters['Birth Month'] = pd.DatetimeIndex(sprinters['DOB']).month

# Find the most common birth month for sprinters in each country
common_birth_month = sprinters.groupby('Country')['Birth Month'].agg(lambda x: x.value_counts().index[0]).reset_index()

fig = px.choropleth(common_birth_month,
                    locations="Country",
                    locationmode='country names',
                    color="Birth Month",
                    hover_name="Country",
                    title="Most Common Birth Month for Sprinters by Country")

fig.show()
#fig.write_html("birth_month_distribution_country_map.html")
#fig.write_image("birth_month_distribution_country_map.svg")

In [ ]:
# Extract the birth year and decade
sprinters['Birth Year'] = pd.DatetimeIndex(sprinters['DOB']).year
sprinters['Decade'] = (sprinters['Birth Year'] // 10) * 10

# Calculate the average 100m PR by country and decade
avg_400m_per_decade = sprinters.groupby(['Country', 'Decade'])['400_PR'].mean().reset_index()

fig = px.choropleth(avg_400m_per_decade,
                    locations="Country",
                    locationmode='country names',
                    color="400_PR",
                    animation_frame="Decade",  # Animate through decades
                    hover_name="Country",
                    color_continuous_scale="Viridis",
                    title="Average 400m Personal Records by Decade")

fig.show()
#fig.write_html("400m_sprinter_performance_by_decade_country_map.html")
#fig.write_image("400m_sprinter_performance_by_decade_country_map.svg")

# ***Statistical Analysis 400m***

In [ ]:
from scipy.stats import zscore
from sklearn.linear_model import LinearRegression

Linear Regression

In [ ]:
# Function to calculate standard deviation (consistency)
df_400m['consistency'] = df_400m['Avg_Season_Best_400m']  # Placeholder: You could replace with actual std per year data if available

### 1. Regression Analysis ###
# Linear regression: relationship between average_time and years_competed
X = df_400m[['Years_Competed_400m', '400_PR']]
y = df_400m['Avg_Season_Best_400m']

# Fit model
model = LinearRegression()
model.fit(X, y)

# Predictions and residuals
df_400m['predicted_time'] = model.predict(X)
df_400m['residuals'] = df_400m['Avg_Season_Best_400m'] - df_400m['predicted_time']

print("Regression coefficients (slope):", model.coef_)
print("Intercept:", model.intercept_)


Z-Score Standardization

In [ ]:
# Z-score for average_time and years_competed
df_400m['z_time'] = zscore(df_400m['Avg Season Best 400m'])
df_400m['z_years'] = zscore(df_400m['Years_Competed_400m'])
df_400m['z_PR'] = zscore(df_400m['400_PR'])
df_400m['z_career_avg_season'] = zscore(df['Career_Avg_400m'])
df_400m['z_total_races'] = zscore(df['total_races_400m'])
df_400m['z_races_per_year'] = zscore(df['races_per_year_400m'])

# Z-score comparison (combine time and years)
df_400m['z_combined'] = (df_400m['z_time'] + df_400m['z_years'] + df_400m['z_PR']+ df_400m['z_career_avg_season'] + df_400m['z_total_races'] + df_400m['z_races_per_year']) / 6

Efficiency / Ratio Analysis

In [ ]:
# Efficiency score (average_time per year competed)
df_400m['efficiency_score'] = df_400m['Avg Season Best 400m'] / df_400m['Years_Competed_400m']

# Efficiency score: How close the sprinter's average season best to their personal best
df_400m['efficiency_score_pr'] = df_400m['400_PR'] / df_400m['Avg Season Best 400m']

# Efficiency score: How close the sprinter's average is to their personal best
df_400m['efficiency_score_pr'] = df_400m['400_PR'] / df_400m['Career_Avg_400m']

Ranking System

In [ ]:
# Combine rankings based on average_time, consistency, and longevity (years_competed)
df_400m['rank_personal_best'] = df_400m['400_PR'].rank(ascending=True)  # Lower personal best is better
df_400m['rank_average_sb'] = df_400m['Avg Season Best 400m'].rank(ascending=True)  # Lower is better
df_400m['rank_career_avg'] = df_400m['Career_Avg_400m'].rank(ascending=True)
#df_400m['rank_consistency'] = df_400m['consistency'].rank(ascending=True)  # Lower std dev is better
df_400m['rank_consistency'] = df_400m['consistency'].abs().rank(ascending=True) # Lower residuals (consistency) is better
df_400m['rank_years_competed'] = df_400m['Years_Competed_400m'].rank(ascending=False)  # Longer careers are better
df_400m['rank_total_races'] = df_400m['total_races_400m'].rank(ascending=False)  # More races is better
df_400m['rank_races_per_year'] = df_400m['Avg_Races_Year_400m'].rank(ascending=False)  # More races per year is better

#Final ranking
df_400m['final_rank'] = df_400m[['rank_personal_best','rank_average_sb', 'rank_career_avg', 'rank_consistency', 'rank_years_competed', 'rank_total_races','rank_races_per_year']].mean(axis=1)

Scatter Plot Visualization

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Years_Competed_400m', y='Avg_Season_Best_400m', data=df_400m, s=100, hue='final_rank', palette='coolwarm')
plt.title('Years Competed vs. Career Average 100m Time')
plt.xlabel('Years Competed')
plt.ylabel('Average 400m Time (s)')
plt.show()

#plt.savefig('SB_avg_400_vs_yrs_competed_ranked.png', format='png', dpi=300)
#plt.savefig('SB_avg_400_vs_yrs_competed_ranked.jpg', format='jpg', dpi=300)

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Years_Competed_400m', y='Career_Avg_400m', data=df_400m, s=100, hue='final_rank', palette='coolwarm')
plt.title('Years Competed vs. Career Average 400m Time')
plt.xlabel('Years Competed')
plt.ylabel('Career Average 400m Time (s)')
plt.show()

#plt.savefig('Career_avg_400_vs_yrs_competed_ranked.png', format='png', dpi=300)
#plt.savefig('Career_avg_400_vs_yrs_competed_ranked.jpg', format='jpg', dpi=300)

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Years_Competed_400m', y='400_PR', data=df_400m, s=100, hue='final_rank', palette='coolwarm')
plt.title('Years Competed vs. 400m PR')
plt.xlabel('Years Competed')
plt.ylabel('400m PR (s)')
plt.show()

#plt.savefig('400_PR_vs_yrs_competed_ranked.png', format='png', dpi=300)
#plt.savefig('400m_PR_vs_yrs_competed_ranked.jpg', format='jpg', dpi=300)

Interactive Scatter Plot

In [ ]:
# @title Years Competed vs. Career Average 400m Time

df_400m = px.data.iris()
fig = px.scatter(df_400m, x="Years_Competed_400m", y="Avg Season Best 400m", color="final_rank",
                 size='Years_Competed_400m', hover_data=['Avg Season Best 400m']) #Potentially switch out career average for personal record regarding hover data.
fig.show()

#fig.write_html("avg_400_vs_yrs_competed_ranked.html")
#fig.write_image("avg_400_vs_yrs_competed_ranked.svg")

In [ ]:
# @title Years Competed vs. Career Average 400m Time

df_400m = px.data.iris()
fig = px.scatter(df_400m, x="Years_Competed_400m", y="Career_Avg_400m", color="final_rank",
                 size='Years_Competed_400m', hover_data=['Career_Avg_400m']) #Potentially switch out career average for personal record regarding hover data.
fig.show()

#fig.write_html("Career_avg_400_vs_yrs_competed_ranked.html")
#fig.write_image("Career_avg_400_vs_yrs_competed_ranked.svg")

In [ ]:
# @title Years Competed vs. 400m PR

df_400m = px.data.iris()
fig = px.scatter(df_400m, x="Years_Competed_400m", y="400_PR", color="final_rank",
                 size='Years_Competed_400m', hover_data=['400_PR'])
fig.show()

#fig.write_html("400m_PR_vs_yrs_competed_ranked.html")
#fig.write_image("400m_PR_vs_yrs_competed_ranked.svg")

In [ ]:
print(df_400m[['Athlete', 'Avg_Season_Best_400m', 'Years_Competed_400m', '400_PR', 'residuals', 'z_combined', 'efficiency_score','efficiency_score_sb', 'efficiency_score_pr' 'final_rank']])

In [ ]:
df_400m_stat_analysis = df_400m[['Athlete', 'Avg_Season_Best_400m', 'Years_Competed_400m', '400_PR', 'residuals', 'z_combined', 'efficiency_score','efficiency_score_sb', 'efficiency_score_pr' 'final_rank']]

In [ ]:
df_400m_stat_analysis = df_400m_stat_analysis.sort_values(by='final_rank')
df_400m_stat_analysis.head(3)

In [ ]:
#What sample size of the dataframe to we want to make into a figure factory table
df_400m_stat_analysis_sample = df_400m_stat_analysis[1:10]

#Cusomize Colors (Add colorscale=colorscale in parentheses of ff.create table)
#colorscale = [[0, '#4d004c'],[.5, '#f2e5ff'],[1, '#ffffff']]
#Cusomize Font Colors (Add font_colors=font in parentheses of ff.create table)
#font=['#FCFCFC', '#00EE00', '#008B00', '#004F00', '#660000', '#CD0000', '#FF3030']

table_data = df_400m_stat_analysis


fig =  ff.create_table(df_400m_stat_analysis_sample)
fig.show()

#fig.write_html("df_400m_stat_analysis_sample_ff.html")
#fig.write_image("df_400m_stat_analysis_sample_ff.svg")


In [ ]:
fig =  ff.create_table(df_400m_stat_analysis)
fig.show()

#fig.write_html("df_400m_stat_analysis_ff.html")
#fig.write_image("df_400m_stat_analysis_ff.svg")

In [ ]:
fig =  ff.create_table(df_400m)
fig.show()

#fig.write_html("df_400m_ff.html")
#fig.write_image("df_400m_ff.svg")

In [ ]:
df_400m = pd.DataFrame(df_400m)

In [ ]:
df_400m_stat_analysis = pd.DataFrame(df_400m_stat_analysis)

# **Analysis of Personal Best Data (PB/PR)**

In [ ]:
sprinters['400_PR'].mean()

In [ ]:
sprinters['400_PR'].median()

In [ ]:
sprinters['400_PR'].max()

In [ ]:
sprinters.nlargest(1,'400_PR')

In [ ]:
sprinters.nsmallest(1,'400_PR')

In [ ]:
sprinters['400_PR'].max()- sprinters['400_PR'].min()


Variance and Standard Devivation


In [ ]:
sprinters['400_PR_deviation'] = sprinters['400_PR'] - sprinters['400_PR'].mean()
sprinters.head()

In [ ]:
sprinters['400_PR'].std()


In [ ]:
sprinters['400_PR_deviation'].mean()


In [ ]:
sprinters['squared_400_PR_deviation'] = sprinters['400_PR_deviation']**2
sprinters

Population Standard Deviation



In [ ]:
np.sqrt(sprinters['squared_400_PR_deviation'].mean())

In [ ]:
sprinters['400_PR'].var(ddof = 0)


In [ ]:
sprinters['400_PR'].std(ddof = 0)


In [ ]:
plt.figure(figsize = (10,6))

std_plot(sprinters['400_PR'], edgecolor = 'black', linewidth = 2)

z-scores

In [ ]:
sprinters['400_PR_z-score'] = (sprinters['400_PR'] - sprinters['400_PR'].mean()) / sprinters['400_PR'].std(ddof = 0)


In [ ]:
sprinters['400_PR_z-score'].std()


Let's look at height z-scores for Usain Bolt

In [ ]:
sprinters.loc[(sprinters.Athlete == 'Usain Bolt')]


Quartiles and Quantiles/Percentiles


In [ ]:
sprinters['400_PR'].quantile(q = 0.25)


In [ ]:
sprinters['400_PR'].quantile(q = 0.5)


In [ ]:
sprinters['400_PR'].quantile(q = 0.75)


In [ ]:
sprinters['400_PR'].describe()


Interquartile Range



In [ ]:
sprinters['400_PR'].quantile(q = 0.75) - sprinters['400_PR'].quantile(q = 0.25)


In [ ]:
plt.figure(figsize = (10,6))

iqr_plot(sprinters['400_PR'], bins = 25, edgecolor = 'black', linewidth = 2)

Observing Outliers in the Dataset



In [ ]:
plt.figure(figsize = (10,6))
sns.boxplot(x = sprinters['400_PR']);

# **Analysis of Season Best Data**

In [ ]:
SB_400m = sprinters_df2[sprinters_df2['Event'] == '400m']

In [ ]:
SB_400m['Time'].mean()

In [ ]:
SB_400m['Time'].median()

In [ ]:
SB_400m['Time'].max()

In [ ]:
SB_400m.nlargest(1,'Time')

In [ ]:
SB_400m.nsmallest(1,'Time')

In [ ]:
SB_400m['Time'].max()- SB_400m['Time'].min()


Variance and Standard Devivation


In [ ]:
SB_400m['400m_SB_deviation'] = SB_400m['Time'] - SB_400m['Time'].mean()
SB_400m.head()

In [ ]:
SB_400m['Time'].std()


In [ ]:
SB_400m['400m_SB_deviation'].mean()


In [ ]:
SB_400m['squared_400m_SB_deviation'] = SB_400m['400m_SB_deviation']**2
SB_400m

Population Standard Deviation



In [ ]:
np.sqrt(SB_400m['squared_400m_SB_deviation'].mean())

In [ ]:
SB_400m['Time'].var(ddof = 0)


In [ ]:
SB_400m['Time'].std(ddof = 0)


In [ ]:
plt.figure(figsize = (10,6))

std_plot(SB_400m['Time'], edgecolor = 'black', linewidth = 2)

z-scores

In [ ]:
SB_400m['100m_SB_z-score'] = (SB_400m['Time'] - SB_400m['Time'].mean()) / SB_400m['Time'].std(ddof = 0)


In [ ]:
SB_400m['400m_SB_z-score'].std()


Let's look at height z-scores for Usain Bolt

In [ ]:
SB_400m.loc[(SB_400m.Athlete == 'Usain Bolt')]


Quartiles and Quantiles/Percentiles


In [ ]:
SB_400m['Time'].quantile(q = 0.25)


In [ ]:
SB_400m['Time'].quantile(q = 0.5)


In [ ]:
SB_400m['Time'].quantile(q = 0.75)


In [ ]:
SB_400m['Time'].describe()


Interquartile Range



In [ ]:
SB_400m['Time'].quantile(q = 0.75) - SB_400m['Time'].quantile(q = 0.25)


In [ ]:
plt.figure(figsize = (10,6))

iqr_plot(SB_400m['Time'], bins = 25, edgecolor = 'black', linewidth = 2)

Observing Outliers in the Dataset



In [ ]:
plt.figure(figsize = (10,6))
sns.boxplot(x = SB_400m['Time']);

**Statisical Tests**

*ANOVA test comparing event times across athletes*

This test will check if there are statistically significant differences in the average times for different athletes in a specific event.

It will identify whether the differences in mean times between athletes are greater than would be expected by random chance.

400m ANOVA

In [ ]:
# Create a list of times for each athlete
athlete_SB_times_400 = [group['Time'].values for name, group in SB_400m.groupby('Athlete')]

# Run one-way ANOVA
f_stat, p_val = stats.f_oneway(*athlete_times_400)
print(f"ANOVA result: F-statistic = {f_stat}, p-value = {p_val}")


*T-test - Compare Two Athletes' Performances*



We will use the T-test to T-test to compare the performance of two athletes in a particular event. This will assess if the difference between the two athletes' performance is statistically significant.

400m T-test

In [ ]:
# Filter data for two specific athletes in the 400m event
athlete_400_SB_a = sprinters_df2[(sprinters_df2['Athlete'] == 'Athlete A') & (sprinters_df2['Event'] == '400m')]['Time']
athlete_400_SB_b = sprinters_df2[(sprinters_df2['Athlete'] == 'Athlete B') & (sprinters_df2['Event'] == '400m')]['Time']

# Run independent t-test
t_stat, p_val = stats.ttest_ind(athlete_400_SB_a, athlete_400_SB_b)
print(f"T-test result: T-statistic = {t_stat}, p-value = {p_val}")


*Correlation Analysis – Time vs. Year for a Specific Event*

This will allow us to explore whether there's a trend in athletes' performance over time by checking the correlation between Year and Time in different events.

400m Correlation Analysis

In [ ]:
correlation_400_SB = SB_400m['Year'].corr(SB_400m['Time'])
print(f"Correlation between Year and Time (400m): {correlation}")


*Time Series Analysis – Track Athlete Performance Over Time*

Let's perform time series analysis to track an individual athlete’s performance. This will allow us to detect patterns, trends, or seasonal effects in an athlete's performance over time.

In [ ]:
# Filter data for one athlete
athlete_data_SB = sprinters_df2[sprinters_df2['Athlete'] == 'Athlete A'].sort_values(by='Year')

# Calculate the rolling average (moving average) for the time over 3 events
athlete_data_SB['Moving_Avg'] = athlete_data_SB['Time'].rolling(window=3).mean()

# Plot the moving average
athlete_data_SB[['Year', 'Moving_Avg']].plot(x='Year', y='Moving_Avg')


*Linear Regression – Predict Time Based on Year and Other Variables*



We can use linear regression to predict race times based on year, athlete, location, or other factors. This can help us model how times change over time or in different conditions.

*Linear Regression 400m*

In [ ]:
# Prepare the data (for example, predicting time in the 400m event)
event_data_SB = sprinters_df2[sprinters_df2['Event'] == '400m']
X = event_data_SB[['Year']]  # You can add other features such as 'Location', 'Athlete'
y = event_data_SB['Time']

# Fit the linear regression model
model = LinearRegression()
model.fit(X, y)

# Print the coefficients and intercept
print(f"Coefficient: {model.coef_}, Intercept: {model.intercept_}")


Logistic Regression Model

In [ ]:
logreg = sm.logit('Year ~ Time', data = event_data_SB).fit()

In [ ]:
logreg.summary()

In [ ]:
logreg.predict(event_data_SB)

Logistic Regression Interference

In [ ]:
logreg_full = sm.logit('Year ~ Time + Insert_Column_Here', data = event_data_SB).fit()
logreg_reduced = sm.logit('Year ~ Time', data = event_data_SB).fit()

In [ ]:
logreg_full.llf

In [ ]:
logreg_reduced.llf

In [ ]:
G2 = -2 * (logreg_reduced.llf - logreg_full.llf)
G2

Chi Squared Distribution

In [ ]:
df = logreg_full.df_model - logreg_reduced.df_model

In [ ]:
chi2.sf(G2, df = df)

Let's consense the logistic regression interference and chi-squared distribution into one cell.

In [ ]:
logreg_full = sm.logit('Year ~ Time + Insert_Column_Here', data = event_data_SB).fit()
logreg_reduced = sm.logit('Year ~ Time', data = event_data_SB).fit()

G2 = -2 * (logreg_reduced.llf - logreg_full.llf)
df = logreg_full.df_model - logreg_reduced.df_model
print(f'p-value: {chi2.sf(G2, df = df)}')

Generating Predictions

We'll stratify to ensure that the proportion of the stratify variable is the same in the training data and in the test data.

In [ ]:
SB_train, SB_test = train_test_split(event_data_SB, test_size = 0.25, stratify = event_data_CD['Year'], random_state = 321)

We'll fit a model using several of our predictor variables.

In [ ]:
logreg_pred = sm.logit('Year ~ Time + Insert + Columns + Here + In + These + Blank + Spaces + Check + Dataframe',
                       data = event_data_SB).fit()

In [ ]:
y_pred = logreg_pred.predict(SB_test) > 0.5

Let's see how well the predicted values matche up to the true labels.

In [ ]:
pd.crosstab(SB_test['Year'], y_pred)

In [ ]:
cm_analysis(SB_test['Year'], y_pred, labels = [0, 1], figsize = (7,6))

Visualizations

In [ ]:
SB_400m['Time'].hist();

In [ ]:
probplot(SB_400m['Time'], plot=plt);

In [ ]:
plt.figure(figsize = (10,6))

std_plot(SB_400m['Time'], edgecolor = 'black', linewidth = 2)

In [ ]:
plt.figure(figsize = (10,6))

iqr_plot(SB_400m['Time'], bins = 25, edgecolor = 'black', linewidth = 2)

In [ ]:
plt.figure(figsize = (10,6))
sns.boxplot(x = CD_400m['Time']);

# **Analysis of Career Data**

In [ ]:
CD_400m = sprinters_df3[sprinters_df3['Event'] == '400m']

In [ ]:
CD_400m['Time'].mean()

In [ ]:
CD_400m['Time'].median()

In [ ]:
CD_400m['Time'].max()

In [ ]:
CD_400m.nlargest(1,'Time')

In [ ]:
CD_400m.nsmallest(1,'Time')

In [ ]:
CD_400m['Time'].max()- CD_400m['Time'].min()


Variance and Standard Devivation


In [ ]:
CD_400m['400m_CD_deviation'] = CD_400m['Time'] - CD_400m['Time'].mean()
CD_400m.head()

In [ ]:
CD_400m['Time'].std()


In [ ]:
CD_400m['400m_CD_deviation'].mean()


In [ ]:
CD_400m['squared_400m_CD_deviation'] = CD_400m['400m_CD_deviation']**2
CD_400m

Population Standard Deviation



In [ ]:
np.sqrt(CD_400m['squared_400m_CD_deviation'].mean())

In [ ]:
CD_400m['Time'].var(ddof = 0)


In [ ]:
CD_400m['Time'].std(ddof = 0)


In [ ]:
plt.figure(figsize = (10,6))

std_plot(CD_400m['Time'], edgecolor = 'black', linewidth = 2)

z-scores

In [ ]:
CD_400m['400m_CD_z-score'] = (CD_400m['Time'] - CD_400m['Time'].mean()) / CD_400m['Time'].std(ddof = 0)


In [ ]:
CD_400m['400m_CD_z-score'].std()


Let's look at height z-scores for Usain Bolt

In [ ]:
CD_400m.loc[(CD_100m.Athlete == 'Usain Bolt')]


Quartiles and Quantiles/Percentiles


In [ ]:
CD_400m['Time'].quantile(q = 0.25)


In [ ]:
CD_400m['Time'].quantile(q = 0.5)


In [ ]:
CD_400m['Time'].quantile(q = 0.75)


In [ ]:
CD_400m['Time'].describe()


Interquartile Range



In [ ]:
CD_400m['Time'].quantile(q = 0.75) - CD_400m['Time'].quantile(q = 0.25)


In [ ]:
plt.figure(figsize = (10,6))

iqr_plot(CD_400m['Time'], bins = 25, edgecolor = 'black', linewidth = 2)

Observing Outliers in the Dataset



In [ ]:
plt.figure(figsize = (10,6))
sns.boxplot(x = CD_400m['Time']);

Visualizations

In [ ]:
CD_400m['Time'].hist();

In [ ]:
probplot(CD_400m['Time'], plot=plt);

In [ ]:
plt.figure(figsize = (10,6))

std_plot(CD_400m['Time'], edgecolor = 'black', linewidth = 2)

In [ ]:
plt.figure(figsize = (10,6))

iqr_plot(CD_400m['Time'], bins = 25, edgecolor = 'black', linewidth = 2)

In [ ]:
plt.figure(figsize = (10,6))
sns.boxplot(x = CD_400m['Time']);

**Statisical Tests**

400m ANOVA

In [ ]:
# Create a list of times for each athlete
athlete_SB_times_400 = [group['Time'].values for name, group in CD_400m.groupby('Athlete')]

# Run one-way ANOVA
f_stat, p_val = stats.f_oneway(*athlete_times_400)
print(f"ANOVA result: F-statistic = {f_stat}, p-value = {p_val}")


400m T-test

In [ ]:
# Filter data for two specific athletes in the 100m event
athlete_400_CD_a = sprinters_df3[(sprinters_df3['Athlete'] == 'Athlete A') & (sprinters_df3['Event'] == '400m')]['Time']
athlete_400_CD_b = sprinters_df3[(sprinters_df3['Athlete'] == 'Athlete B') & (sprinters_df3['Event'] == '400m')]['Time']

# Run independent t-test
t_stat, p_val = stats.ttest_ind(athlete_400_CD_a, athlete_400_CD_b)
print(f"T-test result: T-statistic = {t_stat}, p-value = {p_val}")


100m Correlation Analysis

In [ ]:
correlation_400_CD = SB_400m['Year'].corr(CD_400m['Time'])
print(f"Correlation between Year and Time (400m): {correlation}")


*Time Series Analysis – Track Athlete Performance Over Time*

In [ ]:
# Filter data for one athlete
athlete_data_CD = sprinters_df3[sprinters_df3['Athlete'] == 'Athlete A'].sort_values(by='Year')

# Calculate the rolling average (moving average) for the time over 3 events
athlete_data_CD['Moving_Avg'] = athlete_data_CD['Time'].rolling(window=3).mean()

# Plot the moving average
athlete_data_CD[['Year', 'Moving_Avg']].plot(x='Year', y='Moving_Avg')


*Linear Regression – Predict Time Based on Year and Other Variables*



*Linear Regression 400m*

In [ ]:
# Prepare the data (for example, predicting time in the 400m event)
event_data_CD = sprinters_df3[sprinters_df3['Event'] == '400m']
X = event_data_CD[['Year']]  # You can add other features such as 'Location', 'Athlete'
y = event_data_CD['Time']

# Fit the linear regression model
model = LinearRegression()
model.fit(X, y)

# Print the coefficients and intercept
print(f"Coefficient: {model.coef_}, Intercept: {model.intercept_}")


Logistic Regression Model

In [ ]:
logreg = sm.logit('Year ~ Time', data = event_data_CD).fit()

In [ ]:
logreg.summary()

In [ ]:
logreg.predict(event_data_CD)

Logistic Regression Interference

In [ ]:
logreg_full = sm.logit('Year ~ Time + Insert_Column_Here', data = event_data_CD).fit()
logreg_reduced = sm.logit('Year ~ Time', data = event_data_CD).fit()

In [ ]:
logreg_full.llf

In [ ]:
logreg_reduced.llf

In [ ]:
G2 = -2 * (logreg_reduced.llf - logreg_full.llf)
G2

Chi Squared Distribution

In [ ]:
df2 = logreg_full.df_model - logreg_reduced.df_model

In [ ]:
chi2.sf(G2, df2 = df2)

Let's consense the logistic regression interference and chi-squared distribution into one cell.

In [ ]:
logreg_full = sm.logit('Year ~ Time + Insert_Column_Here', data = event_data_CD).fit()
logreg_reduced = sm.logit('Year ~ Time', data = event_data_CD).fit()

G2 = -2 * (logreg_reduced.llf - logreg_full.llf)
df2 = logreg_full.df_model - logreg_reduced.df_model
print(f'p-value: {chi2.sf(G2, df2 = df2)}')

Generating Predictions

We'll stratify to ensure that the proportion of the stratify variable is the same in the training data and in the test data.

In [ ]:
CD_train, CD_test = train_test_split(event_data_CD, test_size = 0.25, stratify = event_data_CD['Year'], random_state = 321)

We'll fit a model using several of our predictor variables.

In [ ]:
logreg_pred = sm.logit('Year ~ Time + Insert + Columns + Here + In + These + Blank + Spaces + Check + Dataframe',
                       data = event_data_CD).fit()

In [ ]:
y_pred = logreg_pred.predict(CD_test) > 0.5

Let's see how well the predicted values matche up to the true labels.

In [ ]:
pd.crosstab(CD_test['Year'], y_pred)

In [ ]:
cm_analysis(CD_test['Year'], y_pred, labels = [0, 1], figsize = (7,6))

**Update these and place them where necessary (If necessary)**

Single Linear Regrssion

In [ ]:
lr = sm.ols(
    formula = 'Insert_variable_1 ~ Insert_variable_2',
    data = Insert_df
).fit()

In [ ]:
lr.summary()

In [ ]:
x_pred = pd.DataFrame({'Insert_variable_2': np.linspace(start = Insert_df['Insert_variable_2'].min(),
                                             stop = Insert_df['Insert_variable_2'].max(),
                                             num = 250)
                      })

pred = lr.predict(x_pred)

Insert_df.plot(
    kind = 'scatter',
    x = 'Insert_variable_2',
    y = 'Insert_variable_1',
    figsize = (10,6)
)

plt.plot(x_pred['Insert_variable_2'], pred, color = 'black');

In [ ]:
lr.rsquared

Let's verify that this is the correct $R^2$ value.

To compute TSS, we need to look at the difference between the target values and the average value of the target variable.

In [ ]:
tss = ((Insert_df['Insert_variable_1'] - Insert_df['Insert_variable_1'].mean())**2).sum()

For RSS, we need to consider the difference between the target and the predicted value.

In [ ]:
rss = ((Insert_df['Insert_variable_1'] - lr.fittedvalues)**2).sum()

Now, we can verify that we get the same result

In [ ]:
(tss - rss) / tss

In [ ]:
lr.pvalues['Insert_variable_2']

In [ ]:
lr.conf_int(0.05)

In [ ]:
Insert_df['Insert_variable_2'].describe()

Predictions using the linear regression model

In [ ]:
lr.predict(pd.DataFrame({'Insert_variable_2': [91]})) #Enter time in Square brackets

In [ ]:
lr.get_prediction(pd.DataFrame({'Insert_variable_2': [91]})).conf_int()

To get a prediction interval, which tells us what we can expect for a new observation, we can specify obs = True.

In [ ]:
lr.get_prediction(pd.DataFrame({'Insert_variablel_2': [91]})).conf_int(obs = True)

Let's get all these predictions in a summary frame method

In [ ]:
lr.get_prediction(pd.DataFrame({'Insert_variable_2': [91]})).summary_frame()

Multi-Linear Regression Models

In [ ]:
lr_reduced = sm.ols('Insert_variable_1 ~ Insert_variable_2', data = possum).fit()
lr_full = sm.ols('Insert_variable_1 ~ Insert_variable_2 + Insert_variable_3', data = possum).fit()

In [ ]:
stats.stats.anova_lm(lr_reduced, lr_full)

Interactions in the multi-linear regression model

In [ ]:
lr_full =sm.ols('Insert_variable_1 ~ Insert_variable_2 + Insert_variable_a1 + Insert_variable_2:sex', data = possum).fit()
lr_full.summary()

In [ ]:
lr_reduced =sm.ols('Insert_variable_1 ~ Insert_variable_2 + Insert_variable_a1', data = possum).fit()
lr_full =sm.ols('Insert_variable_1 ~ Insert_variable_2 + Insert_variable_a1 + Insert_variable_2:sex', data = possum).fit()

stats.stats.anova_lm(lr_reduced, lr_full)

In [ ]:
lr_df = sm.ols('variable_a ~ variable_b', data = cars).fit()

plt.figure(figsize = (10,6))
plt.scatter(df['variable_b'], lr_df.resid)
xmin, xmax = plt.xlim()
plt.hlines(y = 0, xmin = xmin, xmax = xmax)
plt.xlim(xmin, xmax);

In [ ]:
var = 'variable_b'

x_pred = pd.DataFrame({
    var: np.linspace(start = Insert_df[var].min(),
                               stop = Insert_df[var].max(), num = 250)
})

pred = lr_poly_log.get_prediction(x_pred).summary_frame()

Insert_df.plot(kind = 'scatter', x = var, y = 'variable_a', figsize = (10,6))

plt.plot(x_pred[var], np.exp(pred['mean']), color = 'grey', label = 'predicted mean')

plt.plot(x_pred[var], np.exp(pred['mean_ci_lower']), color = 'blue', label = 'confidence interval')
plt.plot(x_pred[var], np.exp(pred['mean_ci_upper']), color = 'blue')

plt.plot(x_pred[var], np.exp(pred['obs_ci_lower']), color = 'black', label = 'prediction interval')
plt.plot(x_pred[var], np.exp(pred['obs_ci_upper']), color = 'black')

plt.legend();

# **Let's put all the data frames created into an excel workbook**

In [ ]:
xlwriter = pd.ExcelWriter('400M_Analysis.xlsx')
df_400m.to_excel(xlwriter, sheet_name='400m')
df_400m_stat_analysis.to_excel(xlwriter, sheet_name='400m Statisitcal Analysis')
xlwriter.close()